# Capstone Project I
# Individual Health Insurance Market Place

In [1]:
# Import libraries 
import pandas as pd
import numpy as np
from collections import Counter
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
#import sys
#import matplotlib
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 150)

## Read FIPS Population, Median Income, Unemployment and Health Insurance Marketplace Enrollees in 2017

We use the Area Health Resource File (AHRF) downloaded from the Health Resource and Services Administration website. The file combines health and other socioeconomic data sources on specific geographic areas in the US. I downloaded the set of files with measures at the County level.

The data file used here was directly downloaded from the HRSA website (AHRF2019.asc). The data dictionary was created using the SAS program provided to read the ASCII file. See program AHRF_Dictionary.ipynb.
Source: https://data.hrsa.gov/data/download - Area Health Resources Files (AHRF) 2018-2019. US Department of Health and Human Services, Health Resources and Services Administration, Bureau of Health Workforce, Rockville, MD.

In [2]:
# Set paths and file names
datapath='C:\\Users\\l_gas\\Documents\\Development\\SpringBoard_DataScience\\HealthInsuranceData'
data2019= datapath + "\\AHRF\\AHRF_2018-2019\\DATA\\AHRF2019.asc"

In [3]:
# Import Area Research File data - Median household income (2017), Health insurance market place enrollees (2017)
# Identify variables
ahrfdata = pd.read_csv('AHRF2019_dict.csv',index_col=0)
var_selection = ahrfdata[(ahrfdata.Label.str.contains("FIPS")) | (ahrfdata.Label.str.contains("CBSA")) | (ahrfdata.Name == "f12424") | 
                         (ahrfdata.Name == "f00008") | (ahrfdata.Label == "Median Household Income 2017") | 
                         (ahrfdata.Label == "Hlth Ins Marketplace Enrollees 2017") | (ahrfdata.Label == "Unemployment Rate, 16+ 2017") | 
                         (ahrfdata.Label == "Population Estimate 2017") | (ahrfdata.Label == "Total Number Hospitals 2017")] 
position = [(var_selection.iloc[i,0]-1,int(var_selection.iloc[i,3])) for i in range(len(var_selection))]
print(var_selection)

# Import selected columns
area_char = pd.read_fwf(data2019,colspecs = position, header=None)
area_char.columns = ['fips', 'state', 'st','fips_st','fips_county', 'CBSA', 'CBSA_Name', 'CBSA_Ind', 'CBSA_Status','Num_Hosp2017','Pop2017','MedIncome2017','IndMarketEnroll2017','Unemployment']

area_char['Num_Hosp2017'] = area_char['Num_Hosp2017'].apply(pd.to_numeric, errors='coerce')
area_char['Pop2017'] = area_char['Pop2017'].apply(pd.to_numeric, errors='coerce')
area_char['MedIncome2017'] = area_char['MedIncome2017'].apply(pd.to_numeric, errors='coerce')
area_char['IndMarketEnroll2017'] = area_char['IndMarketEnroll2017'].apply(pd.to_numeric, errors='coerce')
# Unemployment rate is in percentage form and has one decimal implied (see documentation)
area_char['Unemployment'] = area_char['Unemployment'].apply(pd.to_numeric, errors='coerce')/10


      Pos_Start      Name  String  Pos_End                                              Label
1             2    f00002    True      6.0                      Header - FIPS St and Cty Code
7            46    f00008    True     64.0                                         State Name
8            65    f12424    True     66.0                            State Name Abbreviation
11          122    f00011    True    123.0                                    FIPS State Code
12          124    f00012    True    126.0                                   FIPS County Code
19          167  f1389118    True    171.0  Core Based Stat Area Code(CBSA) Metropolitan/M...
20          172  f1389218    True    221.0  Core Based Stat Area Name(CBSA) Metropolitan/M...
21          222  f1406718    True    222.0  CBSA Indicator Code 0 = Not, 1 = Metro, 2 = Mi...
22          223  f1419518    True    230.0        CBSA County Status Central or Outlying 2018
3413      11456  f0886817   False  11458.0                  

In [4]:
## Check there are unique entries per each FIPS code - Print summary of dataframe with variable selection.

print("Number of Lines in the area_char file: ",len(area_char))
print("Number of distinct FIPS code entries in the area_char file: ",len(Counter(area_char['fips'])))
print("Descriptive Measures from the area_char file (includes ALL FIPS):")
print(area_char.describe())


Number of Lines in the area_char file:  3230
Number of distinct FIPS code entries in the area_char file:  3230
Descriptive Measures from the area_char file (includes ALL FIPS):
               fips      fips_st  fips_county          CBSA     CBSA_Ind  Num_Hosp2017       Pop2017  MedIncome2017  IndMarketEnroll2017  \
count   3230.000000  3230.000000  3230.000000   1916.000000  3220.000000   3224.000000  3.221000e+03    3141.000000          2719.000000   
mean   31423.921981    31.320743   103.178947  29872.829332     0.801242      1.947581  1.022116e+05   51090.529449          3384.237955   
std    16380.125272    16.365500   107.218177  11561.589118     0.756210      4.120137  3.291528e+05   13497.965734         13418.161762   
min     1001.000000     1.000000     1.000000  10100.000000     0.000000      0.000000  8.800000e+01   22679.000000             0.000000   
25%    19029.500000    19.000000    35.000000  19340.000000     0.000000      1.000000  1.117300e+04   42275.000000        

## Read Health Insurance Individual Market Plans - Federal Mandated Markets

The Centers for Medicare & Medicaid Services uploads health insurance plan information on all federally facilitated marketplace through the healthcare.gov website. For this project we use the 2018 QHP landscape data (https://www.healthcare.gov/health-plan-information-2018/).

We import the Individual Market data for 2018 and combine with AHRF extracted data.

In [5]:
# Import Health Insurance Individual Market Plans data - Only columns relevant for the analysis are imported.

plans_2018 = pd.read_csv(r'C:\Users\l_gas\Documents\Development\SpringBoard_DataScience\HealthInsuranceData\IndivMarketPlans\QHP_PY2018_Medi-_Indi-_Land.csv',
                   low_memory=False, usecols=['State Code','FIPS County Code','County Name',
                'Metal Level','Issuer Name','HIOS Issuer ID','Plan ID (Standard Component)','Plan Marketing Name','Plan Type',
                'Standardized Plan Design','Rating Area','Child Only Offering','Source','Accreditation',
                'Adult Dental ','Child Dental '])
plans_2018 = plans_2018.rename(columns={"State Code": "State", "FIPS County Code": "fips", "HIOS Issuer ID": "Issuer_ID"})


In [6]:
print(plans_2018.head())

  State  fips     County Name Metal Level                               Issuer Name  Issuer_ID Plan ID (Standard Component)  \
0    AK  2013  Aleutians East        Gold  Premera Blue Cross Blue Shield of Alaska      38344               38344AK0540003   
1    AK  2013  Aleutians East      Silver  Premera Blue Cross Blue Shield of Alaska      38344               38344AK0540006   
2    AK  2013  Aleutians East      Bronze  Premera Blue Cross Blue Shield of Alaska      38344               38344AK0540008   
3    AK  2013  Aleutians East      Bronze  Premera Blue Cross Blue Shield of Alaska      38344               38344AK0540009   
4    AK  2013  Aleutians East      Silver  Premera Blue Cross Blue Shield of Alaska      38344               38344AK0540010   

                                 Plan Marketing Name Plan Type Standardized Plan Design    Rating Area          Child Only Offering Source  \
0        Premera Blue Cross Preferred Plus Gold 1500       PPO           Not Applicable  Rating

#### Number of plans by Accreditation Flag

In [7]:
accreditation = plans_2018.groupby('Accreditation')[['Plan ID (Standard Component)']].agg(['count'])
print("Number of Plans by Accreditation Flag", accreditation)


Number of Plans by Accreditation Flag               Plan ID (Standard Component)
                                     count
Accreditation                             
NCQA                                 16235
URAC                                  3092


#### Number of plans by Metal Level

In [8]:
metal_lvl = plans_2018.groupby('Metal Level')[['Plan ID (Standard Component)']].agg(['count'])
print("Number of Plans by Metal Level", metal_lvl)

Number of Plans by Metal Level                 Plan ID (Standard Component)
                                       count
Metal Level                                 
Bronze                                  9211
Catastrophic                            2819
Expanded Bronze                         3008
Gold                                    6847
Platinum                                 564
Silver                                 16899


#### Group data by CBSA (FIPS) code

In [9]:
# Generate Measures: number of plans offered and number of issuers.

plan_cnt = plans_2018.groupby('fips')[['Plan ID (Standard Component)']].agg(['count'])
plan_cnt.columns = ['count_plans']
plan_inscnt = plans_2018.groupby('fips')[['Issuer_ID']].agg(['nunique'])
plan_inscnt.columns = ['count_issuers']

## Combine Plan Information and FIP Population Measures

In [10]:
plans_ahrf = pd.merge(left=plan_cnt, right=plan_inscnt, left_on='fips', right_on='fips')
plans_ahrf = pd.merge(left=plans_ahrf, right=area_char, left_on='fips', right_on='fips')

In [11]:
print(len(plans_ahrf))
print(plans_ahrf.describe())


2722
               fips  count_plans  count_issuers      fips_st  fips_county          CBSA     CBSA_Ind  Num_Hosp2017       Pop2017  MedIncome2017  \
count   2722.000000  2722.000000    2722.000000  2722.000000  2722.000000   1558.000000  2721.000000   2721.000000  2.721000e+03    2721.000000   
mean   31350.903012    14.455547       1.710507    31.239897   111.005878  29464.891528     0.784638      1.802279  8.348002e+04   49720.981624   
std    14970.806015    10.558827       1.004195    14.952237   112.759340  11401.271870     0.770265      3.652337  2.428415e+05   12610.015798   
min     1001.000000     2.000000       1.000000     1.000000     1.000000  10100.000000     0.000000      0.000000  1.340000e+02   22679.000000   
25%    19135.500000     7.000000       1.000000    19.000000    39.000000  19110.000000     0.000000      1.000000  1.058500e+04   41524.000000   
50%    31016.000000    12.000000       1.000000    31.000000    85.000000  29180.000000     1.000000      1.00000

#### Add State Governor's Political Party Affiliation

We identify the governor's political party at the beginning of 2018 for all fifty states. The data is stored in a csv file containing three columns: state code, governor's political affiliation at the begining of 2018, and an indicator of the individual insurance market being federally facilitated or managed by the state.

In [12]:
# Set paths and file name
projpath='C:\\Users\\l_gas\\Documents\\Development\\SpringBoard_DataScience\\CapstoneProj1'
stparty= projpath + "\\All_States_Governors_Party.csv"
# Read csv file
stparty_all = pd.read_csv(stparty)
fed_stparty =stparty_all[stparty_all['Market_Reg']=='Federal']

In [13]:
plans_data = pd.merge(left=plans_ahrf, right=fed_stparty, left_on='st', right_on='State Code')

## Clean the Analytic File

We look at the list of variables and their type in the final table. We also run a description of each numerical variable using the method 'describe'. There are four county areas with missing values under the individual Market Enrollment column, and one county with missing values under the 2017 Median Income, 2017 Population, and Unemployment columns accounting for a total of four counties with some missing values. Since this is a very small portion of the total number of observations (4/2722 or less than 0.15%), we exclude those four rows from the analysis.
We don't observe any clear outliers in the file.

In [17]:
# Summarize variables
print(plans_data.info())
print(plans_data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2718 entries, 0 to 2717
Data columns (total 19 columns):
fips                   2718 non-null int64
count_plans            2718 non-null int64
count_issuers          2718 non-null int64
state                  2718 non-null object
st                     2718 non-null object
fips_st                2718 non-null int64
fips_county            2718 non-null int64
CBSA                   1556 non-null float64
CBSA_Name              1556 non-null object
CBSA_Ind               2718 non-null float64
CBSA_Status            1556 non-null object
Num_Hosp2017           2718 non-null float64
Pop2017                2718 non-null float64
MedIncome2017          2718 non-null float64
IndMarketEnroll2017    2718 non-null float64
Unemployment           2718 non-null float64
State Code             2718 non-null object
Pparty                 2718 non-null object
Market_Reg             2718 non-null object
dtypes: float64(7), int64(5), object(7)
memory usage: 4

In [15]:
# Print lines with some columns with missing values
print(plans_data[(plans_data.Pop2017.isnull() | plans_data.MedIncome2017.isnull() | plans_data.IndMarketEnroll2017.isnull() | plans_data.Unemployment.isnull())])

# The number of rows with some variables with missing values is very small (<0.15%), so we drop those counties from the analysis
plans_data = plans_data[(plans_data.Pop2017.notnull() & plans_data.MedIncome2017.notnull() & plans_data.IndMarketEnroll2017.notnull() & plans_data.Unemployment.notnull())]
plans_data = plans_data.reset_index(drop=True)
print("Total Number of Rows with no Missing Values",len(plans_data))

       fips  count_plans  count_issuers     state  st  fips_st  fips_county     CBSA           CBSA_Name  CBSA_Ind CBSA_Status  Num_Hosp2017  \
81     2158            5              1    Alaska  AK        2          158      NaN                 NaN       0.0         NaN           0.0   
2305  48301           13              2     Texas  TX       48          301  37780.0  Pecos, TX Micro SA       2.0    Outlying           0.0   
2351  48393           13              2     Texas  TX       48          393  37420.0  Pampa, TX Micro SA       2.0    Outlying           0.0   
2534  51515           10              1  Virginia  VA       51          515      NaN                 NaN       NaN         NaN           NaN   

      Pop2017  MedIncome2017  IndMarketEnroll2017  Unemployment State Code       Pparty Market_Reg  
81     8202.0        31250.0                  NaN          20.1         AK  Independent    Federal  
2305    134.0        65203.0                  NaN           5.0         TX   

In [18]:
# Potential outliers
print(plans_data[plans_data.count_plans>80])

       fips  count_plans  count_issuers      state  st  fips_st  fips_county     CBSA                                        CBSA_Name  CBSA_Ind  \
192   12009          104              4    Florida  FL       12            9  37340.0       Palm Bay-Melbourne-Titusville, FL Metro SA       1.0   
204   12035           92              4    Florida  FL       12           35  19660.0  Deltona-Daytona Beach-Ormond Beach, FL Metro SA       1.0   
246   12117          119              5    Florida  FL       12          117  36740.0           Orlando-Kissimmee-Sanford, FL Metro SA       1.0   
251   12127          111              5    Florida  FL       12          127  19660.0  Deltona-Daytona Beach-Ormond Beach, FL Metro SA       1.0   
1754  39151           85              5       Ohio  OH       39          151  15940.0                    Canton-Massillon, OH Metro SA       1.0   
2676  55105          102              4  Wisconsin  WI       55          105  27500.0                   Janesvil

In [16]:
plans_data.to_json("./plans_data.json", compression=None)